In [0]:
parameters = [
    {
        "table": "db2099.silver.factstreams",
        "alias": "factstreams",
        "cols": ["factstreams.stream_id", "factstreams.listen_duration"]
    },
    {
        "table": "db2099.silver.dimuser",
        "alias": "dimuser",
        "cols": ["dimuser.user_id", "dimuser.user_name"],
        "condition": "factstreams.user_id = dimuser.user_id"
    },
    {
        "table": "db2099.silver.dimtrack",
        "alias": "dimtrack",
        "cols": ["dimtrack.track_id", "dimtrack.track_name"],
        "condition": "factstreams.track_id = dimtrack.track_id"
    }
]


In [0]:
pip install jinja2

In [0]:
from jinja2 import Template

In [0]:
query_text = """
SELECT
    {% for param in parameters %}
        {{ param.cols | join(', ') }}
        {% if not loop.last %}
            ,
        {% endif %}
    {% endfor %}
FROM
    {% for param in parameters %}
        {% if loop.first %}
            {{ param['table'] }} AS {{ param['alias'] }}
        {% endif %}
    {% endfor %}
    {% for param in parameters %}
        {% if not loop.first %}
        LEFT JOIN
            {{ param['table'] }} AS {{ param['alias'] }}
        ON
            {{ param['condition'] }}
        {% endif %}
    {% endfor %}
"""


In [0]:
jinja_sql_str = Template(query_text)
query = jinja_sql_str.render(parameters=parameters)
print(query)

In [0]:
display(spark.sql(query))

In [0]:
spark.sql("SELECT COUNT(*) FROM db2099.silver.factstreams").show()